In [5]:
import pandas as pd
import numpy as np
import os
from pathlib import Path


In [7]:
# -----------------------------------------------------------------------------
DATA_DIR = Path("data/raw")
OUTPUT_FILE = DATA_DIR / "sales_data.csv"
# Filter criteria
STORE_ID = 1              # Focus on one store (can be parameterized later)
FAMILY_CATEGORY = "GROCERY I"  # High-volume, stable category
TARGET_METRIC = "sales"   # We are forecasting revenue

# Ensure directories exist
os.makedirs(DATA_DIR, exist_ok=True)

In [8]:
#-----------------------------------------------------------------------------
# 2. LOAD RAW DATASETS
# -----------------------------------------------------------------------------
print("🔍 Loading raw datasets...")

# Main sales data
train_df = pd.read_csv(
    DATA_DIR / "train.csv",
    parse_dates=["date"],
    dtype={
        "store_nbr": "int8",
        "family": "category",
        "sales": "float32",
        "onpromotion": "uint16"
    }
)

# Holidays
holidays_df = pd.read_csv(
    DATA_DIR / "holidays_events.csv",
    parse_dates=["date"]
)

# Oil prices (economic signal)
oil_df = pd.read_csv(
    DATA_DIR / "oil.csv",
    parse_dates=["date"]
)

# Stores metadata
stores_df = pd.read_csv(DATA_DIR / "stores.csv")

print(f"✅ Loaded {len(train_df):,} sales records")
print(f"✅ Loaded {len(holidays_df):,} holiday records")
print(f"✅ Loaded {len(oil_df):,} oil price records")
print(f"✅ Loaded {len(stores_df):,} store records")


🔍 Loading raw datasets...
✅ Loaded 3,000,888 sales records
✅ Loaded 350 holiday records
✅ Loaded 1,218 oil price records
✅ Loaded 54 store records


In [9]:
# -----------------------------------------------------------------------------
# 3. FILTER TO TARGET STORE & PRODUCT FAMILY
# -----------------------------------------------------------------------------
print(f"\n🎯 Filtering to Store {STORE_ID}, Product Family: '{FAMILY_CATEGORY}'...")

filtered_df = train_df[
    (train_df["store_nbr"] == STORE_ID) &
    (train_df["family"] == FAMILY_CATEGORY)
].copy()

if filtered_df.empty:
    raise ValueError(f"No data found for Store {STORE_ID}, Family '{FAMILY_CATEGORY}'")

print(f"✅ Filtered to {len(filtered_df):,} daily records")

# -----------------------------------------------------------------------------
# 4. RESAMPLE TO MONTHLY REVENUE (MS = Month Start)
# -----------------------------------------------------------------------------
print("\n📅 Resampling to monthly total sales...")

# Set date as index
filtered_df.set_index("date", inplace=True)

# Resample: monthly sum of sales
monthly_df = filtered_df[[TARGET_METRIC]].resample("MS").sum()

# Reset index
monthly_df = monthly_df.reset_index()
monthly_df.rename(columns={TARGET_METRIC: "revenue"}, inplace=True)

print("✅ Monthly aggregation complete")
print(monthly_df.head())



🎯 Filtering to Store 1, Product Family: 'GROCERY I'...
✅ Filtered to 1,684 daily records

📅 Resampling to monthly total sales...
✅ Monthly aggregation complete
        date  revenue
0 2013-01-01  53397.0
1 2013-02-01  45447.0
2 2013-03-01  52736.0
3 2013-04-01  54654.0
4 2013-05-01  55290.0


In [10]:
# 5. ADD HOLIDAY FLAG (Monthly Level)
# -----------------------------------------------------------------------------
print("\n🎄 Adding holiday flags...")

# Extract month-year from holidays
holidays_df["date"] = pd.to_datetime(holidays_df["date"])
holidays_df["year_month"] = holidays_df["date"].dt.to_period("M")
holidays_df = holidays_df[
    (holidays_df["type"] == "Holiday") &
    (holidays_df["transferred"] == False)
]

# Get unique holiday months
holiday_months = holidays_df["year_month"].drop_duplicates()

# Add flag to monthly data
monthly_df["year_month"] = pd.to_datetime(monthly_df["date"]).dt.to_period("M")
monthly_df["is_holiday_month"] = monthly_df["year_month"].isin(holiday_months).astype(int)

# Drop helper column
monthly_df.drop(columns=["year_month"], inplace=True)

print("✅ Holiday flags added")


🎄 Adding holiday flags...
✅ Holiday flags added


In [12]:
# -----------------------------------------------------------------------------
# 6. ADD OIL PRICE (Monthly Average) - Modern Pandas Version
# -----------------------------------------------------------------------------
print("\n🛢️ Merging oil prices (modern pandas style)...")

# Prepare oil data
oil_df["year_month"] = pd.to_datetime(oil_df["date"]).dt.to_period("M")
oil_monthly = oil_df.groupby("year_month")["dcoilwtico"].mean().reset_index()
oil_monthly["year_month"] = pd.PeriodIndex(oil_monthly["year_month"], freq="M")

# Prepare monthly_df
monthly_df["year_month"] = pd.PeriodIndex(pd.to_datetime(monthly_df["date"]).dt.to_period("M"), freq="M")

# Merge
monthly_df = pd.merge(monthly_df, oil_monthly, how="left", on="year_month")
monthly_df.rename(columns={"dcoilwtico": "avg_oil_price"}, inplace=True)
monthly_df.drop(columns=["year_month"], inplace=True)

# Fill missing oil prices — Modern, safe way
monthly_df["avg_oil_price"] = (
    monthly_df["avg_oil_price"]
    .ffill()
    .bfill()
)

print("✅ Oil price merged and cleaned (no deprecated methods)")


🛢️ Merging oil prices (modern pandas style)...
✅ Oil price merged and cleaned (no deprecated methods)


In [13]:
# 7. FINAL CLEANUP & SAVE
# -----------------------------------------------------------------------------
print("\n🧹 Final cleanup and export...")

# Sort by date
monthly_df = monthly_df.sort_values("date").reset_index(drop=True)

# Ensure correct data types
monthly_df["revenue"] = np.round(monthly_df["revenue"], 2)
monthly_df["avg_oil_price"] = np.round(monthly_df["avg_oil_price"], 2)

# Save to CSV
monthly_df[["date", "revenue", "is_holiday_month", "avg_oil_price"]].to_csv(OUTPUT_FILE, index=False)

print(f"✅ Clean dataset saved to {OUTPUT_FILE}")
print(f"📊 Final shape: {monthly_df.shape}")
print("\n📅 First 5 rows:")
print(monthly_df.head())

print("\n✅ DATA PREPROCESSING COMPLETE — READY FOR EDA & FORECASTING")


🧹 Final cleanup and export...
✅ Clean dataset saved to data\raw\sales_data.csv
📊 Final shape: (56, 5)

📅 First 5 rows:
        date  revenue  is_holiday_month  avg_oil_price  avg_oil_price
0 2013-01-01  53397.0                 1          94.76          94.76
1 2013-02-01  45447.0                 1          95.31          95.31
2 2013-03-01  52736.0                 1          92.94          92.94
3 2013-04-01  54654.0                 1          92.02          92.02
4 2013-05-01  55290.0                 1          94.51          94.51

✅ DATA PREPROCESSING COMPLETE — READY FOR EDA & FORECASTING
